In [1]:
import gc
import pandas as pd
import numpy as np
import os
import xgboost as xgb
import lightgbm as lgb
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
# from datetime import date
# def getWeekday(row):
#     if row == 'null':
#         return row
#     else:
#         return date(int(row[0:4]), int(row[4:6]), int(row[6:8])).weekday() + 1

In [2]:
####################训练集#######################
DATA_PATH = "datasets"
csv_path = os.path.join(DATA_PATH, "atec_anti_fraud_train.csv")
data = pd.read_csv(csv_path)

In [ ]:
data.sort_values("date",inplace=True)

In [ ]:
data['weekday'] = data['date'].astype(str).apply(getWeekday)

In [3]:
####################训练集#######################
pd.set_option('display.max_columns',2000)
data

,id,label,date,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f62,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118,f119,f120,f121,f122,f123,f124,f125,f126,f127,f128,f129,f130,f131,f132,f133,f134,f135,f136,f137,f138,f139,f140,f141,f142,f143,f144,f145,f146,f147,f148,f149,f150,f151,f152,f153,f154,f155,f156,f157,f158,f159,f160,f161,f162,f163,f164,f165,f166,f167,f168,f169,f170,f171,f172,f173,f174,f175,f176,f177,f178,f179,f180,f181,f182,f183,f184,f185,f186,f187,f188,f189,f190,f191,f192,f193,f194,f195,f196,f197,f198,f199,f200,f201,f202,f203,f204,f205,f206,f207,f208,f209,f210,f211,f212,f213,f214,f215,f216,f217,f218,f219,f220,f221,f222,f223,f224,f225,f226,f227,f228,f229,f230,f231,f232,f233,f234,f235,f236,f237,f238,f239,f240,f241,f242,f243,f244,f245,f246,f247,f248,f249,f250,f251,f252,f253,f254,f255,f256,f257,f258,f259,f260,f261,f262,f263,f264,f265,f266,f267,f268,f269,f270,f271,f272,f273,f274,f275,f276,f277,f278,f279,f280,f281,f282,f283,f284,f285,f286,f287,f288,f289,f290,f291,f292,f293,f294,f295,f296,f297
0,f10eb20f31cf7063ee8bdbd1272214e4d7e0193c8dbce4...,0,20171103,0,0,0,0,100807.0,0,5,1,1,1,1,2,0,2,0,2,2,2,1,31.0,61.0,142.0,245.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,0.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,3.0,96.0,38.0,301.0,302.0,302.0,302.0,301.0,302.0,40.0,79.0,40.0,79.0,2.0,3.0,4.0,3.0,41.0,80.0,96.5,721.1,3252.0,154.5,394.5,2.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,4.0,8.0,15.0,30.0,4.0,14.0,34.0,70.0,142.0,3.0,15.0,41.0,79.0,191.0,192.0,190.0,190.0,190.0,190.0,191.0,192.0,191.0,190.0,190.0,190.0,32.0,33.0,25.0,25.0,24.0,25.0,31.0,79.0,42.0,42.0,42.0,68.0,32.0,32.0,28.0,27.0,28.0,27.0,32.0,235.0,235.0,126.0,126.0,127.0,126.0,235.0,192.0,191.0,190.0,189.0,189.0,301.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,3.0,3.0,4.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,4.0,3.0,4.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,3.0,3.0,3.0,14.0,41.0,80.0,167.0,2.0,2.0,4.0,3.0,3.0,4.0,6.0,6.0,2.0,2.0,4.0,3.0,2.0,2.0,3.0,4.0,3.0,3.0,4.0,1.0,3.0,3.0,4.0,2.0,2.0,2.0,2.0,8.0,2.0,2.0,3.0,3.0,3.0,2.0,5.0,5.0,10.0,16.0,2.0,2.0,4.0,5.0,7.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,27.0,297.0,302.0,384.0,770.0,20.0,99.0,102.0,124.0,140.0,301.0,312.0,328.0,85.0,302.0,201.0,203.0,203.0,61.0,201.0
1,d861929b67938d06538b910b9f6b85f5eb62b6ad7361ba...,0,20170917,0,1,1,1,100805.0,1,5,2,1,1,0,2,1,2,2,1,1,2,0,14.0,13.0,48.0,48.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0,1.0,1.0,2.0,4.0,4.0,5.0,1.0,1.0,4.0,3.0,3.0,79.0,12.0,302.0,32.0,301.0,302.0,302.0,302.0,23.0,26.0,23.0,26.0,2.0,2.0,3.0,4.0,23.0,26.0,5.0,717.4,733.3,231.8,472.8,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,1.0,4.0,8.0,11.0,12.0,4.0,12.0,23.0,26.0,27.0,5.0,12.0,24.0,27.0,122.0,121.0,16.0,121.0,121.0,122.0,121.0,122.0,121.0,16.0,121.0,122.0,32.0,31.0,13.0,27.0,26.0,27.0,32.0,79.0,40.0,39.0,39.0,63.0,31.0,31.0,15.0,28.0,29.0,29.0,31.0,231.0,232.0,17.0,99.0,99.0,98.0,232.0,121.0,122.0,17.0,121.0,121.0,302.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,4.0,5.0,12.0,24.0,26.0,27.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,3.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1

In [ ]:
data[:5]

In [ ]:
len(set(data['id']))

In [8]:
set(data['date'])

{20170905,
 20170906,
 20170907,
 20170908,
 20170909,
 20170910,
 20170911,
 20170912,
 20170913,
 20170914,
 20170915,
 20170916,
 20170917,
 20170918,
 20170919,
 20170920,
 20170921,
 20170922,
 20170923,
 20170924,
 20170925,
 20170926,
 20170927,
 20170928,
 20170929,
 20170930,
 20171001,
 20171002,
 20171003,
 20171004,
 20171005,
 20171006,
 20171007,
 20171008,
 20171009,
 20171010,
 20171011,
 20171012,
 20171013,
 20171014,
 20171015,
 20171016,
 20171017,
 20171018,
 20171019,
 20171020,
 20171021,
 20171022,
 20171023,
 20171024,
 20171025,
 20171026,
 20171027,
 20171028,
 20171029,
 20171030,
 20171031,
 20171101,
 20171102,
 20171103,
 20171104,
 20171105}

In [4]:
####################测试集#######################
csv_path = os.path.join(DATA_PATH, "atec_anti_fraud_test_a.csv")
data_test = pd.read_csv(csv_path)

In [12]:
data_test.sort_values("date",inplace=True)

In [18]:
data_test['weekday'] = data_test['date'].astype(str).apply(getWeekday)

In [5]:
####################测试集#######################
data_test

,id,date,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f62,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118,f119,f120,f121,f122,f123,f124,f125,f126,f127,f128,f129,f130,f131,f132,f133,f134,f135,f136,f137,f138,f139,f140,f141,f142,f143,f144,f145,f146,f147,f148,f149,f150,f151,f152,f153,f154,f155,f156,f157,f158,f159,f160,f161,f162,f163,f164,f165,f166,f167,f168,f169,f170,f171,f172,f173,f174,f175,f176,f177,f178,f179,f180,f181,f182,f183,f184,f185,f186,f187,f188,f189,f190,f191,f192,f193,f194,f195,f196,f197,f198,f199,f200,f201,f202,f203,f204,f205,f206,f207,f208,f209,f210,f211,f212,f213,f214,f215,f216,f217,f218,f219,f220,f221,f222,f223,f224,f225,f226,f227,f228,f229,f230,f231,f232,f233,f234,f235,f236,f237,f238,f239,f240,f241,f242,f243,f244,f245,f246,f247,f248,f249,f250,f251,f252,f253,f254,f255,f256,f257,f258,f259,f260,f261,f262,f263,f264,f265,f266,f267,f268,f269,f270,f271,f272,f273,f274,f275,f276,f277,f278,f279,f280,f281,f282,f283,f284,f285,f286,f287,f288,f289,f290,f291,f292,f293,f294,f295,f296,f297
0,8e8290c270ec4bc3448dd5edd35c6f059b42d38f9ddd6f...,20180120,0,0,1,1,100809.0,1,3,2,0,1,0,1,1,2,2,1,2,2,1,1.0,15.0,0.0,92.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,2.0,5.0,4.0,1.0,1.0,3.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,3.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,3.0,4.0,4.0,6.0,24.0,60.0,60.0,1.0,2.0,3.0,3.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,3.0,2.0,3.0,2.0,1.0,2.0,2.0,3.0,2.0,2.0,1.0,10.0,11.0,2.0,2.0,1.0,3.0,4.0,2.0,2.0,3.0,12.0,13.0,2.0,1.0,2.0,5.0,7.0,1.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,7.0,1.0,2.0,3.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,0.0,2.0,1.0,1.0,71.0,171.0,301.0,337.0,385.0,31.0,75.0,122.0,130.0,130.0,302.0,302.0,302.0,134.0,302.0,215.0,215.0,215.0,94.0,214.0
1,f36c5fa5c0e7afccf733a4d74c7e06ffe43cc8fd24eda7...,20180105,1,1,0,0,100808.0,1,5,2,2,1,1,1,2,0,1,0,0,0,1,0.0,30.0,1.0,173.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,12.0,13.0,12.0,13.0,13.0,13.0,12.0,2.0,1.0,2.0,1.0,13.0,12.0,12.0,12.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,3.0,3.0,2.0,2.0,2.0,1.0,2.0,3.0,3.0,2.0,3.0,3.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,13.0,13.0,12.0,12.0,13.0,2.0,3.0,2.0,1.0,1.0,1.0,3.0,3.0,1.0,2.0,2.0,1.0,2.0,1.0,77.0,78.0,78.0,77.0,78.0,38.0,39.0,39.0,39.0,39.0,39.0,40.0,40.0,39.0,40.0,39.0,39.0,39.0,39.0,39.0
2,ee85f808b1fd49eaba308527e1686c509dc8e3e5057488...,20180109,0,0,0,0,100803.0,0,6,1,1,0,2,1,2,

In [7]:
len(set(data_test['id']))

491668

In [9]:
set(data_test['date'])

{20180105,
 20180106,
 20180107,
 20180108,
 20180109,
 20180110,
 20180111,
 20180112,
 20180113,
 20180114,
 20180115,
 20180116,
 20180117,
 20180118,
 20180119,
 20180120,
 20180121,
 20180122,
 20180123,
 20180124,
 20180125,
 20180126,
 20180127,
 20180128,
 20180129,
 20180130,
 20180131,
 20180201,
 20180202,
 20180203,
 20180204,
 20180205}

In [20]:
# change weekday to one-hot encoding 
weekdaycols = ['weekday_' + str(i) for i in range(1,8)]
print(weekdaycols)

tmpdf = pd.get_dummies(data['weekday'])
tmpdf.columns = weekdaycols
data[weekdaycols] = tmpdf

tmpdf = pd.get_dummies(data_test['weekday'])
tmpdf.columns = weekdaycols
data_test[weekdaycols] = tmpdf

['weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7']


In [23]:
data.drop("weekday", axis=1, inplace=True)
data_test.drop("weekday", axis=1, inplace=True)

In [6]:
data[data['label'] == 1]

,id,label,date,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f62,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118,f119,f120,f121,f122,f123,f124,f125,f126,f127,f128,f129,f130,f131,f132,f133,f134,f135,f136,f137,f138,f139,f140,f141,f142,f143,f144,f145,f146,f147,f148,f149,f150,f151,f152,f153,f154,f155,f156,f157,f158,f159,f160,f161,f162,f163,f164,f165,f166,f167,f168,f169,f170,f171,f172,f173,f174,f175,f176,f177,f178,f179,f180,f181,f182,f183,f184,f185,f186,f187,f188,f189,f190,f191,f192,f193,f194,f195,f196,f197,f198,f199,f200,f201,f202,f203,f204,f205,f206,f207,f208,f209,f210,f211,f212,f213,f214,f215,f216,f217,f218,f219,f220,f221,f222,f223,f224,f225,f226,f227,f228,f229,f230,f231,f232,f233,f234,f235,f236,f237,f238,f239,f240,f241,f242,f243,f244,f245,f246,f247,f248,f249,f250,f251,f252,f253,f254,f255,f256,f257,f258,f259,f260,f261,f262,f263,f264,f265,f266,f267,f268,f269,f270,f271,f272,f273,f274,f275,f276,f277,f278,f279,f280,f281,f282,f283,f284,f285,f286,f287,f288,f289,f290,f291,f292,f293,f294,f295,f296,f297
125,77c8378c369eac5c76a9fa9dd956775c2bd119864f6390...,1,20171103,1,0,1,1,100803.0,0,3,2,1,0,1,0,2,1,0,1,1,2,1,31.0,60.0,166.0,276.0,2.0,1.0,2.0,2.0,3.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,2.0,1.0,1.0,1.0,6.0,6.0,7.0,6.0,7.0,2.0,1.0,1.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,2.0,3.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,3.0,3.0,1.0,2.0,1.0,3.0,3.0,1.0,2.0,1.0,2.0,3.0,7.0,6.0,6.0,7.0,6.0,29.0,28.0,1.0,2.0,3.0,3.0,7.0,7.0,7.0,8.0,2.0,1.0,2.0,3.0,2.0,2.0,2.0,3.0,1.0,3.0,3.0,2.0,1.0,3.0,4.0,6.0,7.0,7.0,7.0,10.0,1.0,2.0,2.0,2.0,2.0,6.0,6.0,8.0,8.0,16.0,2.0,1.0,3.0,3.0,6.0,1.0,2.0,1.0,1.0,2.0,6.0,6.0,7.0,6.0,28.0,7.0,6.0,26.0,2.0,2.0,3.0,6.0,6.0,2.0,1.0,1.0,3.0,2.0,3.0,44.0,214.0,301.0,394.0,825.0,26.0,99.0,122.0,134.0,139.0,302.0,307.0,317.0,126.0,301.0,217.0,218.0,217.0,98.0,218.0
216,d101ee32bfd62a08b27f3a9450a09dcc0b0266a11391b1...,1,20171009,0,0,1,0,100805.0,0,1,0,0,0,1,2,2,2,1,1,1,2,1,2.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,2.0,1.0,2.0,2.0,13.0,12.0,13.0,12.0,13.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,27.0,18.0,19.0,27.0,26.0,26.0,25.0,25.0,25.0,25.0,17.0,18.0,19.0,27.0,27.0,26.0,2988.0,8889.0,8888.0,8888.0,8889.0,2.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,26.0,26.0,27.0,27.0,27.0,26.0,27.0,27.0,26.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,27.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,2.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,2.0,2.0,1.0,17.0,20.0,26.0,27.0,26.0,26.0,27.0,2.0,1.0,1.0,1.0,18.0,20.0,27.0,27.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,3.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,3.0,2.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,8.0,9.0,8.0,3.0,3.0,3.0,1.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,8.0,9.0,8.0,1.0,1.0,2.0,1.0,2.0,3.0,3.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0
223,48c923ec4efc56447e3738e16a624884006a0537321eb0...,1,201

In [12]:
data[data['id'] == "dcc70d96faaa51bd902e31ae26898770b78d2caa7bba185a2347a908080a6b9b"]

,id,label,date,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f62,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118,f119,f120,f121,f122,f123,f124,f125,f126,f127,f128,f129,f130,f131,f132,f133,f134,f135,f136,f137,f138,f139,f140,f141,f142,f143,f144,f145,f146,f147,f148,f149,f150,f151,f152,f153,f154,f155,f156,f157,f158,f159,f160,f161,f162,f163,f164,f165,f166,f167,f168,f169,f170,f171,f172,f173,f174,f175,f176,f177,f178,f179,f180,f181,f182,f183,f184,f185,f186,f187,f188,f189,f190,f191,f192,f193,f194,f195,f196,f197,f198,f199,f200,f201,f202,f203,f204,f205,f206,f207,f208,f209,f210,f211,f212,f213,f214,f215,f216,f217,f218,f219,f220,f221,f222,f223,f224,f225,f226,f227,f228,f229,f230,f231,f232,f233,f234,f235,f236,f237,f238,f239,f240,f241,f242,f243,f244,f245,f246,f247,f248,f249,f250,f251,f252,f253,f254,f255,f256,f257,f258,f259,f260,f261,f262,f263,f264,f265,f266,f267,f268,f269,f270,f271,f272,f273,f274,f275,f276,f277,f278,f279,f280,f281,f282,f283,f284,f285,f286,f287,f288,f289,f290,f291,f292,f293,f294,f295,f296,f297
